In [ ]:
import os
!pip install uv

# Latests transformers + vision libraries
!uv pip install --system -q -U git+https://github.com/huggingface/transformers
!uv pip install --system -q -U timm torchvision Pillow numpy==1.*
!uv pip install pycocotools
# For training
!uv pip install --system -q -U datasets accelerate
!uv pip install --system -q -U "albumentations==1.4.6" torchmetrics
# checkpoint = "/content/rtdetr-v2-finetune/checkpoint-672"

# parmaeter : 42M


In [1]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f'Using device: {device}')

Using device: cuda


In [2]:
from utils import load_models
checkpoint = "PekingU/rtdetr_r50vd"
tmp = load_models(checkpoint)
image_processor = tmp['image_processor']
train_transform = tmp['train_transformer']
validation_transform = tmp['validation_transformer']
model = tmp['model']
evaluator = tmp['MAPEvaluator']

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Some weights of RTDetrForObjectDetection were not initialized from the model checkpoint at PekingU/rtdetr_r50vd and are newly initialized because the shapes did not match:
- model.decoder.class_embed.0.bias: found shape torch.Size([80]) in the checkpoint and torch.Size([1]) in the model instantiated
- model.decoder.class_embed.0.weight: found shape torch.Size([80, 256]) in the checkpoint and torch.Size([1, 256]) in the model instantiated
- model.decoder.class_embed.1.bias: found shape torch.Size([80]) in the checkpoint and torch.Size([1]) in the model instantiated
- model.decoder.class_embed.1.weight: found shape torch.Size([80, 256]) in the checkpoint and torch.Size([1, 256]) in the model instantiated
- model.decoder.class_embed.2.bias: found shape torch.Size([80]) in the checkpoint and torch.Size([1]) in the model instantiated
- model.decoder.class_embed.2.weight: found shape torch.Size([80, 256]) in the checkpoint and torch.Size([1, 256]) in the model instantiated
- model.decoder.cl

In [3]:
from data_load import loading_data , RTDataset
paths = {
    'train_json':"/root/workspace/data/TL_KS_BBOX",
    'train_jpg':"/root/workspace/data/TS_KS",
    'validation_json':'/root/workspace/data/VL_KS_BBOX',
    'validation_jpg':'/root/workspace/data/VS_KS'
}
train_data = loading_data(paths['train_json'],paths['train_jpg'])
print(train_data[0])
validation_data = loading_data(paths['validation_json'],paths['validation_jpg'])
train_RTdataset = RTDataset(train_data, image_processor, transform=train_transform)
print(train_RTdataset[0])
validation_RTdataset = RTDataset(validation_data, image_processor, transform=validation_transform)


{'image_id': 0, 'image_name': 'K3A_CHN_20161112052404_23', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x7FC0AC4F1070>, 'bboxes': [[15, 114, 72, 18]], 'category': [0]}
1
{'image_id': 0, 'category_id': 0, 'bbox': (np.float64(425.0), np.float64(114.0), np.float64(72.0), np.float64(18.0)), 'iscrowd': 0, 'area': np.float64(1296.0)}
1
{'pixel_values': tensor([[[0.3451, 0.3451, 0.3451,  ..., 0.6784, 0.6706, 0.6667],
         [0.3412, 0.3569, 0.3608,  ..., 0.6706, 0.6745, 0.6824],
         [0.3412, 0.3608, 0.3647,  ..., 0.6627, 0.6706, 0.6824],
         ...,
         [0.2510, 0.2314, 0.2235,  ..., 0.2314, 0.2314, 0.2314],
         [0.2353, 0.2275, 0.2314,  ..., 0.2275, 0.2235, 0.2275],
         [0.2157, 0.2196, 0.2431,  ..., 0.2078, 0.2118, 0.2196]],

        [[0.2941, 0.2941, 0.2941,  ..., 0.5686, 0.5608, 0.5569],
         [0.2902, 0.3059, 0.3059,  ..., 0.5608, 0.5647, 0.5725],
         [0.2902, 0.3098, 0.3137,  ..., 0.5569, 0.5686, 0.5765],
         ...,
     

In [4]:
from transformers import TrainingArguments , Trainer
import torch
from data_load import collate_fn



out_dir = "/workspace/RT_50"
training_args = TrainingArguments(
    output_dir=out_dir,
    num_train_epochs=60,
    max_grad_norm=0.1,
    learning_rate=1e-4,
    warmup_steps=300,
    per_device_train_batch_size=24,
    dataloader_num_workers=2,
    metric_for_best_model="eval_map",
    greater_is_better=True,
    load_best_model_at_end=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    remove_unused_columns=False,
    eval_do_concat_batches=False,
    report_to="tensorboard",
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_RTdataset,
    eval_dataset=validation_RTdataset,
    processing_class=image_processor,
    data_collator=collate_fn,
    compute_metrics=evaluator,
)

trainer.train()

24
24
24
24
24


ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/venv/main/lib/python3.12/site-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/venv/main/lib/python3.12/site-packages/torch/utils/data/_utils/fetch.py", line 50, in fetch
    data = self.dataset.__getitems__(possibly_batched_index)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/venv/main/lib/python3.12/site-packages/datasets/arrow_dataset.py", line 2863, in __getitems__
    batch = self.__getitem__(keys)
            ^^^^^^^^^^^^^^^^^^^^^^
  File "/root/workspace/RT_DETR/data_load.py", line 115, in __getitem__
    transformed = self.transform(image=image, bboxes=boxes, category=categories)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/venv/main/lib/python3.12/site-packages/albume

In [ ]:

model_repo = '/content/drive/My Drive/RT_50/checkpoint-2688'
image_processor = AutoImageProcessor.from_pretrained(model_repo)
model = AutoModelForObjectDetection.from_pretrained(model_repo)
image = Image.open('/content/dataset/val/K3A_CHN_20161112052404_15.jpg')
inputs = image_processor(images=[image], return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
target_sizes = torch.tensor([image.size[::-1]])

result = image_processor.post_process_object_detection(outputs, threshold=0.1, target_sizes=target_sizes)[0]

for score, label, box in zip(result["scores"], result["labels"], result["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
        f"Detected {model.config.id2label[label.item()]} with confidence "
        f"{round(score.item(), 3)} at location {box}"
    )
image_with_boxes = image.copy()
draw = ImageDraw.Draw(image_with_boxes)

for score, label, box in zip(result["scores"], result["labels"], result["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    x, y, x2, y2 = tuple(box)
    draw.rectangle((x, y, x2, y2), outline="red", width=1)
    text_label = model.config.id2label[label.item()]
    draw.text((x, y), f"{text_label} [ {score.item():.2f} ]", fill="blue")

image_with_boxes